In [187]:
import pandas as pd
import numpy as np
from pandas.tseries.offsets import MonthEnd

# LOAD DATA
df = pd.read_csv("data.csv")
loan_account_list = df['Loan-Account'].to_list()
principal_list = df['Principal-M'].to_list()
roi_list = df['Rate-of-interest'].to_list()
emi_list = df['EMI'].to_list()
edd_list = df['Earlier-due-date'].to_list()
# df.head()

In [230]:
#LOAD PART PAYMENT DATA
part_payment = pd.read_csv("pre-payment.csv")
part_payment['REPAY_EFFECTIVE_DATE'] = pd.to_datetime(part_payment['REPAY_EFFECTIVE_DATE'],format="%d-%m-%Y")
part_payment['date'] = pd.DatetimeIndex(part_payment['REPAY_EFFECTIVE_DATE']).day
part_payment['month'] = pd.DatetimeIndex(part_payment['REPAY_EFFECTIVE_DATE']).month
part_payment['year'] = pd.DatetimeIndex(part_payment['REPAY_EFFECTIVE_DATE']).year
#part_payment.head(8)
part_payment.head()

,Loan Account,Receipt Number,PART_PREPAY_AMOUNT,REPAY_EFFECTIVE_DATE,date,month,year
0,FSAPLALONS000005000115,6458210,729,2022-04-11,11,4,2022
1,FSAPLALONS000005000330,6490591,208,2022-04-30,30,4,2022
2,FSAPLALONS000005000407,6574372,866,2022-06-23,23,6,2022
3,FSAPLALONS000005000487,6472702,598,2022-04-20,20,4,2022
4,FSAPLALONS000005000073,6472722,1000,2022-11-07,7,11,2022


In [231]:
# UTILITY FUNCTIONS
def get_part_payment_dict(part_payment_list):
    part_payment_dict = {}

    for line in part_payment_list:
        if line[0] in part_payment_dict:
            # append the new number to the existing array at this slot
            part_payment_dict[line[0]][0].append(line[1])
            part_payment_dict[line[0]][1].append(line[2])
        else:
            # create a new array in this slot
            part_payment_dict[line[0]] = [[line[1]],[line[2]]]
    return part_payment_dict

def days360(start_date, end_date, method_eu=False):
    start_day = start_date.day
    start_month = start_date.month
    start_year = start_date.year
    end_day = end_date.day
    end_month = end_date.month
    end_year = end_date.year

    if (
        start_day == 31 or
        (
            method_eu is False and
            start_month == 2 and (
                start_day == 29 or (
                    start_day == 28 and
                    start_date.is_leap_year is False
                )
            )
        )
    ):
        start_day = 30

    if end_day == 31:
        if method_eu is False and start_day != 30:
            end_day = 1

            if end_month == 12:
                end_year += 1
                end_month = 1
            else:
                end_month += 1
        else:
            end_day = 30

    return (
        end_day + end_month * 30 + end_year * 360 -
        start_day - start_month * 30 - start_year * 360)

def part_payment_dict_processor(part_payment_dict):
    for key in part_payment_dict.keys():
        sum_pay = 0 
        sum_payment = {}
        temp_dict ={}
        if part_payment_dict.get(key):
            unique_month = list(set(part_payment_dict[key][0]))
            for i,value in enumerate(unique_month):
                equal_indexes =[j for j,val in enumerate(part_payment_dict[key][0]) if val==value]
                for k in equal_indexes:
                    sum_pay += part_payment_dict[key][1][k]
                sum_payment[value] = sum_pay
                sum_pay = 0
            temp_dict[key] = [list(sum_payment.keys()),list(sum_payment.values())]
            part_payment_dict.update(temp_dict)

    return part_payment_dict

In [232]:
part_payment_list=part_payment.loc[part_payment['Loan Account']=="FSAPLALONS000005003574",['month','date','PART_PREPAY_AMOUNT']].values.tolist()
part_payment_dict = get_part_payment_dict(part_payment_list)
part_payment_dict = part_payment_dict_processor(part_payment_dict)


In [233]:
part_payment_dict

{4: [[10, 15], [3000, 2000]], 9: [[1], [416]], 6: [[1], [418]]}

In [234]:
# end_of_month_dates = ['2022-01-31', '2022-02-28', '2022-03-31', '2022-04-30', '2022-05-31', '2022-06-30', '2022-07-31', '2022-08-31', '2022-09-30', '2022-10-31', '2022-11-30', '2022-12-31']
# principal = 35701.08
# roi = 26.73
# emi = 7396
# sm = 9
# part_payment_dict = {9:[[1],[416]]}
# part_payment_dict = {9:[[1,15],[416,1000]]}

def part_payment_interest(end_of_month_dates,principal,roi,emi,part_payment_dict,sm):
    start_date = pd.to_datetime(f"{end_of_month_dates[sm-2]}")
    # if there is single part payment in a month
    if len(part_payment_dict.get(sm)[0]) == 1:
        i = 0
        interest_value_1 = principal * (roi/ 36000) * days360(pd.to_datetime(f"{start_date}"), pd.to_datetime(f'2022-{sm}-{part_payment_dict.get(sm)[0][i]}'))
        principal = principal - part_payment_dict.get(sm)[1][i]
        print("FIRST INTEREST",interest_value_1)
        interest_value_2 = principal * (roi/ 36000) * days360(pd.to_datetime(f'2022-{sm}-{part_payment_dict.get(sm)[0][i]}'), pd.to_datetime(f"{end_of_month_dates[sm-1]}"))
        print("SECOND INTEREST",interest_value_2)
        interest_value = interest_value_1 + interest_value_2
        print("ADDED interest_value",interest_value)
        principal_apposition = emi - interest_value
        principal = principal - principal_apposition
        return interest_value, principal, principal_apposition
    else:
        end_date = pd.to_datetime(f'2022-{sm}-{part_payment_dict.get(sm)[0][0]}')
        interest_value = 0
        for i in range(len(part_payment_dict.get(sm)[0])+1):
            interest_value_1 = principal * (roi/ 36000) * days360(pd.to_datetime(f"{start_date}"), pd.to_datetime(f"{end_date}"))
            print("start_date",start_date)
            print("end_date",end_date)
            print("interest_value in loop",interest_value)
            if i==len(part_payment_dict.get(sm)[0]):
                principal = principal - 0
            else:
                if type(part_payment_dict.get(sm)[1][i]) == list:
                    principal = principal - sum(part_payment_dict.get(sm)[1][i])
                else:
                    principal = principal - part_payment_dict.get(sm)[1][i]
            if i==len(part_payment_dict.get(sm)[0])-1:
                end_date = pd.to_datetime(f"{end_of_month_dates[sm-1]}")
                start_date = f"2022-{sm}-{part_payment_dict.get(sm)[0][i]}"
            elif i==len(part_payment_dict.get(sm)[0]):
                end_date = pd.to_datetime(f"{end_of_month_dates[sm-1]}")
            else:
                end_date = pd.to_datetime(f'2022-{sm}-{part_payment_dict.get(sm)[0][i+1]}')
                start_date = f"2022-{sm}-{part_payment_dict.get(sm)[0][i]}"


            interest_value = interest_value + interest_value_1
            print("inside loop",interest_value)
        
        principal_apposition = emi - interest_value
        print("principal_apposition",principal_apposition)
        print("interest_value after loop",interest_value)
        principal = principal - principal_apposition
        return interest_value, principal, principal_apposition
    

In [238]:
# end_of_month_dates = ['2022-01-31', '2022-02-28', '2022-03-31', '2022-04-30', '2022-05-31', '2022-06-30', '2022-07-31', '2022-08-31', '2022-09-30', '2022-10-31', '2022-11-30', '2022-12-31']
# principal = 35701.08
# roi = 26.73
# emi = 7396
# sm = 9
# part_payment_dict = {9:[[1],[416]]}
# part_payment_dict = {9:[[1,15],[416,1000]]}

def april_part_payment_interest(principal,roi,emi,part_payment_dict,sm,due_date):
    start_date = pd.to_datetime(f'2022-04-01')
    sm = 4 
    # if there is single part payment in a april month
    if len(part_payment_dict.get(4)[0]) == 1:
        i = 0
        interest_value_1 = principal * (roi/ 36000) * days360(pd.to_datetime(f"{start_date}"), pd.to_datetime(f'2022-4-{part_payment_dict.get(4)[0][i]}'))
        print("interest_value_1",interest_value_1)
        print("principal",principal)
        print("days360",days360(pd.to_datetime(f"{start_date}"), pd.to_datetime(f'2022-4-{part_payment_dict.get(4)[0][i]}')))

        print("interest_value between 1 april and part payment date",interest_value_1)

        principal = principal - part_payment_dict.get(sm)[1][i]
        interest_value_2 = principal * (roi/ 36000) * days360(pd.to_datetime(f'2022-{sm}-{part_payment_dict.get(sm)[0][i]}'), pd.to_datetime(f'2022-04-{due_date}'))
        print("interest_value_2",interest_value_2)
        print("interest_value between part payment date and due date",interest_value_2)
        interest_value = interest_value_1 + interest_value_2
        print("ADDED interest_value",interest_value)
        return interest_value, principal
    else:
        end_date = pd.to_datetime(f'2022-{sm}-{part_payment_dict.get(sm)[0][0]}')
        interest_value = 0
        for i in range(len(part_payment_dict.get(sm)[0])+1):
            interest_value_1 = principal * (roi/ 36000) * days360(pd.to_datetime(f"{start_date}"), pd.to_datetime(f"{end_date}"))
            print("interest_value_1",interest_value_1)
            print("start_date",start_date)
            print("end_date",end_date)
            if i==len(part_payment_dict.get(sm)[0]):
                principal = principal - 0
            else:
                principal = principal - part_payment_dict.get(sm)[1][i]
            
            if i==len(part_payment_dict.get(sm)[0])-1:
                end_date = pd.to_datetime(f"2022-04-{due_date}")
                start_date = f"2022-{sm}-{part_payment_dict.get(sm)[0][i]}"
            elif i==len(part_payment_dict.get(sm)[0]):
                end_date = pd.to_datetime(f"2022-04-{due_date}")
            else:
                end_date = pd.to_datetime(f'2022-{sm}-{part_payment_dict.get(sm)[0][i+1]}')
                start_date = f"2022-{sm}-{part_payment_dict.get(sm)[0][i]}"


            interest_value = interest_value + interest_value_1
            print("ADDDDDED interest_value",interest_value)
        
        # principal_apposition = emi - interest_value
        # print("principal_apposition",principal_apposition)
        # print("interest_value after loop",interest_value)
        # principal = principal - principal_apposition
        return interest_value, principal
    

In [240]:

end_of_month_dates = ['2022-01-31', '2022-02-28', '2022-03-31', '2022-04-30', '2022-05-31', '2022-06-30', '2022-07-31', '2022-08-31', '2022-09-30', '2022-10-31', '2022-11-30', '2022-12-31']
interest_value_list = []

def april_interest(principal, roi, emi, due_date, march_interest,part_payment_dict):
    #if partpayment date == due date
    if part_payment_dict.get(4):
        april_interest, principal = april_part_payment_interest(principal,roi,emi,part_payment_dict,4,due_date)
        print("apr interest after part payment",round(april_interest, 2))
        interest_value = april_interest + march_interest
        principal_apposition = emi - interest_value
        closing_principal = principal - principal_apposition
        print("apr closing prinicipal after part payment",round(closing_principal, 2))
        print("LAST INTEREST VALUE",interest_value)
        return interest_value,closing_principal
    else:
        april_interest = principal * (roi/ 36000) * days360(pd.to_datetime(f'2022-04-01'), pd.to_datetime(f'2022-04-{due_date}'))
        print("apr interest",round(april_interest, 2))
    # interest_value = april_interest + march_interest


        interest_value_list.append(interest_value)
        print("april_interest",round(interest_value, 2))
        principal_apposition = emi - interest_value
        print("first prinicipal appo",round(principal_apposition, 2))
        closing_principal = principal - principal_apposition
        print("first closing prinicipal",round(closing_principal, 2))
        return interest_value,closing_principal


def next_values(principal, roi, emi, date, sm, em, part_payment_dict):

    if sm <= 6:
        interest_value = principal * (roi/ 36000) * days360(pd.to_datetime(f'2022-{sm}-{date}'), pd.to_datetime(f'2022-{em}-{date}'))
        # print("days360 value",days360(pd.to_datetime(f'2022-{sm}-{date}'), pd.to_datetime(f'2022-{em}-{date}')))
        # print("sm, em", sm, em)
        # print("date",date)
        print("interest value", round(interest_value,2))
        interest_value_list.append(interest_value)
        principal_apposition = emi - interest_value
        print("principal apposition", round(principal_apposition,2))
        principal = principal - principal_apposition
        print("closing principal", round(principal,2))
        if sm==6:
            # print("closing principal inside june before interest", round(principal,2))
            interest_value = principal * (roi/ 36000) * days360(pd.to_datetime(f'2022-6-{date}'), pd.to_datetime(f'2022-6-30'))
            interest_value_list.append(interest_value)
            print("inside june interest", round(interest_value,2))
            principal = principal + interest_value
            print("june closing principal", round(principal,2))
    if sm>6:

        # print("closing principal after june before interest", round(principal,2))
        if part_payment_dict.get(sm):
            print("INSIDE PART PAYMENT sm, em", sm, em)
            print("principal before part payment", round(principal,2))
            interest_value, principal, principal_apposition = part_payment_interest(end_of_month_dates,principal,roi,emi,part_payment_dict,sm)
            print("principal after part payment", round(principal,2))
            print("principal apposition after part payment", round(principal_apposition,2))
            print("interest value after part payment", round(interest_value,2))
            

        else:
            interest_value = principal * (roi/ 36000) * days360(pd.to_datetime(f"{end_of_month_dates[sm-2]}"), pd.to_datetime(f"{end_of_month_dates[sm-1]}"))

            #print("days360 value",days360(pd.to_datetime(f'2022-{sm}-{date}'), pd.to_datetime(f'2022-{em}-{date}')))
            print("sm, em", sm, em)
            #print("date",date)
            print("interest value", round(interest_value,2))
            interest_value_list.append(interest_value)

            principal_apposition = emi - interest_value
            print("principal apposition", round(principal_apposition,2))
            principal = principal - principal_apposition
            print("closing principal", round(principal,2))
        
    return interest_value,principal

In [241]:
#APRIL MONTH INTEREST
march_interest = 2000
part_payment_dict = get_part_payment_dict(part_payment_list)
part_payment_dict = part_payment_dict_processor(part_payment_dict)
interest,principal = april_interest(principal_list[2], roi_list[2], emi_list[2],edd_list[2],march_interest,part_payment_dict)

# MAY TO SEPTEMBER MONTH INTEREST
for i in range(5, 10):
    interest_value, closing_principal_2 = next_values(principal, roi_list[2], emi_list[2], edd_list[2], i, i+1, part_payment_dict)
    principal = closing_principal_2

interest_value_1 438.724100925
start_date 2022-04-01 00:00:00
end_date 2022-04-10 00:00:00
ADDDDDED interest_value 438.724100925
interest_value_1 232.598111625
start_date 2022-4-10
end_date 2022-04-15 00:00:00
ADDDDDED interest_value 671.32221255
interest_value_1 0.0
start_date 2022-4-15
end_date 2022-04-15 00:00:00
ADDDDDED interest_value 671.32221255
apr interest after part payment 671.32
apr closing prinicipal after part payment 55928.01
LAST INTEREST VALUE 2671.32221255
interest value 1245.8
principal apposition 6150.2
closing principal 49777.81
interest value 1108.8
principal apposition 6287.2
closing principal 43490.61
inside june interest 484.38
june closing principal 43974.99
sm, em 7 8
interest value 979.54
principal apposition 6416.46
closing principal 37558.53
sm, em 8 9
interest value 836.62
principal apposition 6559.38
closing principal 30999.15
INSIDE PART PAYMENT sm, em 9 10
principal before part payment 30999.15
FIRST INTEREST 23.016865221129308
SECOND INTEREST 658.5315